In [1]:
import pandas as pd
import featuretools as ft

In [2]:
df = pd.read_csv("Superstore.csv",encoding='latin1')

In [3]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
# 1. Customers Table
customers_df = df[['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region']].drop_duplicates(subset=['Customer ID']).reset_index(drop=True)

# 2. Products Table
products_df = df[['Product ID', 'Category', 'Sub-Category', 'Product Name']].drop_duplicates(subset=['Product ID']).reset_index(drop=True)

# 3. Orders Table
orders_df = df[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode']].drop_duplicates().reset_index(drop=True)

# 4. OrderDetails Table
orderDetails_df = df[['Order ID', 'Product ID', 'Customer ID', 'Sales', 'Quantity', 'Discount', 'Profit']].drop_duplicates(subset=['Order ID', 'Product ID']).reset_index(drop=True)
orderDetails_df['OrderDetail ID'] = orderDetails_df['Order ID'].astype(str) + "_" + orderDetails_df['Product ID'].astype(str)

In [5]:
# CREATE EMPTY ENTITY SET
es = ft.EntitySet(id="Superstore")

# ADD DATA INTO ENTITY SET
customers_entity = es.add_dataframe(dataframe=customers_df, dataframe_name='Customers',  index='Customer ID')
products_entity = es.add_dataframe(dataframe=products_df,  dataframe_name='Products',  index='Product ID'  )
orders_entity = es.add_dataframe(dataframe=orders_df,  dataframe_name='Orders',  index='Order ID')
orderDetails_entity = es.add_dataframe(dataframe=orderDetails_df,  dataframe_name='OrderDetails',  index='OrderDetail ID')

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\woodwork\type

In [6]:
# DEFINE RELATIONSHIP
relationships = [
    ('Customers', 'Customer ID', 'OrderDetails', 'Customer ID'),
    ('Products', 'Product ID', 'OrderDetails', 'Product ID'),
    ('Orders', 'Order ID', 'OrderDetails', 'Order ID')
]

es.add_relationships(relationships)

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\featuretools\entityset\entityset.py:393: UserWarning: Logical type Categorical for child column Customer ID does not match parent column Customer ID logical type Unknown. Changing child logical type to match parent.
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\featuretools\entityset\entityset.py:393: UserWarning: Logical type Categorical for child column Product ID does not match parent column Product ID logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


Entityset: Superstore
  DataFrames:
    Customers [Rows: 793, Columns: 8]
    Products [Rows: 1862, Columns: 4]
    Orders [Rows: 5009, Columns: 4]
    OrderDetails [Rows: 9986, Columns: 8]
  Relationships:
    OrderDetails.Customer ID -> Customers.Customer ID
    OrderDetails.Product ID -> Products.Product ID
    OrderDetails.Order ID -> Orders.Order ID

In [7]:
# APPLY DFS
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="OrderDetails",
    verbose=True,
    max_depth=2
)


Built 97 features
Elapsed: 00:03 | Progress: 100%|██████████


In [8]:
feature_matrix

,Sales,Quantity,Discount,Profit,Customers.Segment,Customers.Country,Customers.State,Customers.Postal Code,Customers.Region,Products.Category,...,Orders.SUM(OrderDetails.Quantity),Orders.SUM(OrderDetails.Sales),Orders.DAY(Order Date),Orders.DAY(Ship Date),Orders.MONTH(Order Date),Orders.MONTH(Ship Date),Orders.WEEKDAY(Order Date),Orders.WEEKDAY(Ship Date),Orders.YEAR(Order Date),Orders.YEAR(Ship Date)
OrderDetail ID,,,,,,,,,,,,,,,,,,,,,
CA-2016-152156_FUR-BO-10001798,261.9600,2,0.00,41.9136,Consumer,United States,Kentucky,42420,South,Furniture,...,5.0,993.9000,8,11,11,11,1,4,2016,2016
CA-2016-152156_FUR-CH-10000454,731.9400,3,0.00,219.5820,Consumer,United States,Kentucky,42420,South,Furniture,...,5.0,993.9000,8,11,11,11,1,4,2016,2016
CA-2016-138688_OFF-LA-10000240,14.6200,2,0.00,6.8714,Corporate,United States,California,90036,West,Office Supplies,...,2.0,14.6200,12,16,6,6,6,3,2016,2016
US-2015-108966_FUR-TA-10000577,957.5775,5,0.45,-383.0310,Consumer,United States,Florida,33311,South,Furniture,...,7.0,979.9455,11,18,10,10,6,6,2015,2015
US-2015-108966_OFF-ST-10000760,22.3680,2,0.20,2.5164,Consumer,United States,Florida,33311,South,Office Supplies,...,7.0,979.9455,11,18,10,10,6,6,2015,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CA-2014-110422_FUR-FU-10001889,25.2480,3,0.20,4.1028,Consumer,United States,Washington,98105,West,Furniture,...,3.0,25.2480,21,23,1,1,1,3,2014,2014
CA-2017-121258_FUR-FU-10000747,91.9600,2,0.00,15.6332,Consumer,United States,Washington,98115,West,Furniture,...,8.0,380.1360,26,3,2,3,6,4,2017,2017
CA-2017-121258_TEC-PH-10003645,258.5760,2,0.20,19.3932,Consumer,United States,Washington,98115,West,Technology,...,8.0,380.1360,26,3,2,3,6,4,2017,2017


In [9]:
feature_defs

[<Feature: Sales>,
 <Feature: Quantity>,
 <Feature: Discount>,
 <Feature: Profit>,
 <Feature: Customers.Segment>,
 <Feature: Customers.Country>,
 <Feature: Customers.State>,
 <Feature: Customers.Postal Code>,
 <Feature: Customers.Region>,
 <Feature: Products.Category>,
 <Feature: Products.Sub-Category>,
 <Feature: Orders.Ship Mode>,
 <Feature: Customers.COUNT(OrderDetails)>,
 <Feature: Customers.MAX(OrderDetails.Discount)>,
 <Feature: Customers.MAX(OrderDetails.Profit)>,
 <Feature: Customers.MAX(OrderDetails.Quantity)>,
 <Feature: Customers.MAX(OrderDetails.Sales)>,
 <Feature: Customers.MEAN(OrderDetails.Discount)>,
 <Feature: Customers.MEAN(OrderDetails.Profit)>,
 <Feature: Customers.MEAN(OrderDetails.Quantity)>,
 <Feature: Customers.MEAN(OrderDetails.Sales)>,
 <Feature: Customers.MIN(OrderDetails.Discount)>,
 <Feature: Customers.MIN(OrderDetails.Profit)>,
 <Feature: Customers.MIN(OrderDetails.Quantity)>,
 <Feature: Customers.MIN(OrderDetails.Sales)>,
 <Feature: Customers.SKEW(OrderDe

In [10]:
selected_features_df = feature_matrix[[
    "Customers.COUNT(OrderDetails)",
    "Products.COUNT(OrderDetails)", "Products.MAX(OrderDetails.Profit)", "Products.MEAN(OrderDetails.Discount)",
    "Products.SUM(OrderDetails.Quantity)", "Products.SUM(OrderDetails.Sales)",
    "Orders.COUNT(OrderDetails)", "Orders.MAX(OrderDetails.Quantity)",
    "Orders.MEAN(OrderDetails.Profit)", "Orders.SUM(OrderDetails.Sales)",
    "Orders.MONTH(Order Date)", "Orders.YEAR(Order Date)"
]]


In [11]:
selected_features_df = selected_features_df.rename(columns={
    "Customers.COUNT(OrderDetails)": "Customer_Order_Count",
    "Products.COUNT(OrderDetails)": "Product_Order_Count",
    "Products.MAX(OrderDetails.Profit)": "Product_Max_Profit",
    "Products.MEAN(OrderDetails.Discount)": "Product_Mean_Discount",
    "Products.SUM(OrderDetails.Quantity)": "Product_Total_Quantity",
    "Products.SUM(OrderDetails.Sales)": "Product_Total_Sales",
    "Orders.COUNT(OrderDetails)": "Order_Item_Count",
    "Orders.MAX(OrderDetails.Quantity)": "Order_Max_Quantity",
    "Orders.MEAN(OrderDetails.Profit)": "Order_Mean_Profit",
    "Orders.SUM(OrderDetails.Sales)": "Order_Total_Sales",
    "Orders.MONTH(Order Date)": "Order_Month",
    "Orders.YEAR(Order Date)": "Order_Year"
})


In [12]:
selected_features_df = selected_features_df.reset_index()

In [13]:
# Merge the dataframes
orderDetails_df = orderDetails_df.merge(
    selected_features_df,
    on="OrderDetail ID",
    how="left"
)

# Drop the original 'OrderDetail ID' column
orderDetails_df = orderDetails_df.drop(columns=["OrderDetail ID"])
